In [12]:
# Preprocessing steps
# Load Nepali stopwords
# You may need to download a Nepali stopword list
import spacy
import re
import json

# Extract stop words for nepali texts
nepali_stopwords = []
with open('dataset/non-potential-topic-word-list.txt', 'r', encoding='utf-8') as f:
    nepali_stopwords = [line.strip() for line in f]  # Use a set for efficient lookup
    
from sklearn.feature_extraction import text

# Combine with built-in English stopwords
combined_stopwords = list(text.ENGLISH_STOP_WORDS.union(nepali_stopwords))

In [13]:
output_dir = 'results/LDA_NER/'

# Open and read the content of data.txt
with open('dataset/docs_saved_ner.txt', 'r', encoding='utf-8') as file:
    content = file.read()

# If the content is already a valid JSON list, just load it
import json

try:
    docs = json.loads(content)
except json.JSONDecodeError:
    # If not, try to manually split paragraphs (fallback)
    docs = [para.strip() for para in content.split('\n') if para.strip()]

# Print the list or process it further
print(docs[0:10])

['यस रिट निवेदन मिसिल संलग्न कागजात अध्ययन गरी निवेदक उपस्थित हुनुभए विद्वान् अधिवक्ता श्री श्री श्री श्री श्री श्री श्री श्री श्री श्री श्री निवेदक स्वयं पूर्ण राजवंशी गर्नु बहस सुनियो यस के कसो हो निवेदक मागबमोजिम आदेश किन जारी हुनु नपर्ने हो मागबमोजिम आदेश जारी हुनु नपर्ने कुनै आधार कारण भए सबुद प्रमाण म्याद सूचना पाए मिति बाटा म्यादबाहेक दिन विपक्षी महान्यायाधिवक्ता कार्यालय लिखित जवाफ पेस गर्नु भनी आदेश निवेदन प्रतिलिपि साथै राखी विपक्षी नाम म्याद सूचना जारी गरी लिखित जवाफ परे वा अवधि नाघेपछि नियमानुसार गरी पेस गर्नु प्रस्तुत विषय व्यवस्थापकीय पक्ष अन्तरिम आदेश माग सम्बन्ध विचार गर्दा निवेदक जिकिर नाजायज पनि देखिदैन भने सरकार पनि संवेदनशील नै रहे देखिन्छ निवेदक उठाए कोभिड कोरोना भाइरस सङ्क्रमण भयावह कुनै एक भूगोल मात्र सीमित रहे विषय नरहे स्पष्ट यस जोगिन सङ्क्रमण देश भित्रिनै नदिन विपपूर्व जोखिम व्यवस्थापन समयमै हुन अति आवश्यक लोक कल्याणकारी प्रजातान्त्रिक व्यवस्था सरकार देश जनताप्रति उत्तरदायी जिम्मेवार त्यसप्रति गम्भीर रहे नै हुने गर्छ यस केही शं गरिरहनु पर्दैन यस रोकथाम विपपूर्

In [14]:
import gensim
from gensim import corpora
import nltk


# Tokenize and clean
texts = [
    [word for word in doc.split() if word not in combined_stopwords and len(word) > 1]
    for doc in docs
]

In [15]:
# Create dictionary and corpus
dictionary = corpora.Dictionary(texts)

# Filter extremes (optional, tuneable)
dictionary.filter_extremes(no_below=5, no_above=0.5)

corpus = [dictionary.doc2bow(text) for text in texts]

In [16]:
from gensim.models.ldamodel import LdaModel

# Train LDA model
lda_model = LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=100,
    random_state=100,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    per_word_topics=True
)

In [17]:
# Print top 10 words from each topic
topics = lda_model.print_topics(num_words=10)
for i, topic in topics:
    print(f"Topic {i}: {topic}")

Topic 23: 0.000*"भवितव्य" + 0.000*"मुसलमान" + 0.000*"जगाउने" + 0.000*"झुठ्ठा" + 0.000*"पूर्वरिसइवी" + 0.000*"फर्किआए" + 0.000*"मुक्" + 0.000*"एकआपस" + 0.000*"हिर्काए" + 0.000*"भूमिहार"
Topic 90: 0.000*"भवितव्य" + 0.000*"मुसलमान" + 0.000*"जगाउने" + 0.000*"झुठ्ठा" + 0.000*"पूर्वरिसइवी" + 0.000*"फर्किआए" + 0.000*"मुक्" + 0.000*"एकआपस" + 0.000*"हिर्काए" + 0.000*"भूमिहार"
Topic 2: 0.000*"भवितव्य" + 0.000*"मुसलमान" + 0.000*"जगाउने" + 0.000*"झुठ्ठा" + 0.000*"पूर्वरिसइवी" + 0.000*"फर्किआए" + 0.000*"मुक्" + 0.000*"एकआपस" + 0.000*"हिर्काए" + 0.000*"भूमिहार"
Topic 5: 0.000*"भवितव्य" + 0.000*"मुसलमान" + 0.000*"जगाउने" + 0.000*"झुठ्ठा" + 0.000*"पूर्वरिसइवी" + 0.000*"फर्किआए" + 0.000*"मुक्" + 0.000*"एकआपस" + 0.000*"हिर्काए" + 0.000*"भूमिहार"
Topic 9: 0.000*"भवितव्य" + 0.000*"मुसलमान" + 0.000*"जगाउने" + 0.000*"झुठ्ठा" + 0.000*"पूर्वरिसइवी" + 0.000*"फर्किआए" + 0.000*"मुक्" + 0.000*"एकआपस" + 0.000*"हिर्काए" + 0.000*"भूमिहार"
Topic 56: 0.000*"भवितव्य" + 0.000*"मुसलमान" + 0.000*"जगाउने" + 0.000*"झुठ्ठा" 

In [18]:
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(
    model=lda_model,
    texts=texts,
    dictionary=dictionary,
    coherence='c_v'
)


In [19]:
import pandas as pd

topic_words = []
for i, topic in lda_model.show_topics(formatted=False):
    words = [word for word, prob in topic]
    topic_words.append({"Topic": i, "Words": ", ".join(words)})

df = pd.DataFrame(topic_words)
df.to_csv(output_dir + "lda_topics.csv", index=False)

In [20]:
def calculate_topic_diversity(lda_model, top_k=10):
    topic_words = lda_model.show_topics(num_topics=-1, num_words=top_k, formatted=False)

    all_words = []
    for topic in topic_words:
        words = [word for word, _ in topic[1]]
        all_words.extend(words)

    unique_words = set(all_words)
    diversity_score = len(unique_words) / len(all_words)
    return diversity_score

In [21]:

coherence_score = coherence_model_lda.get_coherence()
print(f"Coherence Score: {coherence_score:.4f}")

diversity = calculate_topic_diversity(lda_model, top_k=10)
print(f"Topic Diversity: {diversity:.4f}")

Coherence Score: 0.4302
Topic Diversity: 0.5710


In [22]:
topics[0]

(23,
 '0.000*"भवितव्य" + 0.000*"मुसलमान" + 0.000*"जगाउने" + 0.000*"झुठ्ठा" + 0.000*"पूर्वरिसइवी" + 0.000*"फर्किआए" + 0.000*"मुक्" + 0.000*"एकआपस" + 0.000*"हिर्काए" + 0.000*"भूमिहार"')